In [2]:
import pulp

# Create 6-node power system example
# Node 1: Generator
# Node 2: Load (demand = 90)
# Node 3: Generator
# Node 4: Load (demand = 100)
# Node 5: Generator
# Node 6: Load (demand=125)
# Lines: 1-2, 2-3, 3-4, 4-5, 5-6, 6-1

# Create the model
model = pulp.LpProblem(name="power_system", sense=pulp.LpMinimize)

# Create variables
# Generation variables with bounds
p1 = pulp.LpVariable('p1', lowBound=10, upBound=250)
p2 = pulp.LpVariable('p2', lowBound=10, upBound=300)
p3 = pulp.LpVariable('p3', lowBound=10, upBound=270)


# Line flow variables with bounds
f12 = pulp.LpVariable('f12', lowBound=-50, upBound=50)
f23 = pulp.LpVariable('f23', lowBound=-60, upBound=60)
f34 = pulp.LpVariable('f34', lowBound=-90, upBound=90)
f45 = pulp.LpVariable('f45', lowBound=-50, upBound=50)
f56 = pulp.LpVariable('f56', lowBound=-120, upBound=120)
f61 = pulp.LpVariable('f61', lowBound=-100, upBound=100)

# Phase angle variables
theta1 = pulp.LpVariable('theta1', lowBound=None, upBound=None, cat='Continuous')
theta2 = pulp.LpVariable('theta2', lowBound=None, upBound=None, cat='Continuous')
theta3 = pulp.LpVariable('theta3', lowBound=None, upBound=None, cat='Continuous')
theta4 = pulp.LpVariable('theta4', lowBound=None, upBound=None, cat='Continuous')
theta5 = pulp.LpVariable('theta5', lowBound=None, upBound=None, cat='Continuous')
theta6 = pulp.LpVariable('theta6', lowBound=None, upBound=None, cat='Continuous')

# Fix theta1 as reference
model += theta1 == 0

# Objective function: minimize generation cost
# Cost coefficients: c1 = 5, c2 = 2, c3 = 3
model += 5*p1 + 2*p2 + 3*p3

# Flow conservation constraints
model += p1 - f12 + f61 == 0, "node1_balance"  # Generator node 1
model += -90 + f12 - f23 == 0, "node2_balance" # Load node 2 
model += p2 - f34 + f23 == 0, "node3_balance"  # Generator node 3
model += -100 + f34 - f45 == 0, "node4_balance"  # Load node 4
model += p3 - f56 + f45 == 0, "node5_balance"  # Generator node 5
model += -125 + f56 - f61 ==0 , "node6_balance" # Load node 6

# DC power flow constraints 
model += f12 - 11.6*(theta1 - theta2) == 0, "dc_flow_12"
model += f23 - 5.9*(theta2 - theta3) == 0, "dc_flow_23"
model += f34 - 13.7*(theta3 - theta4) == 0, "dc_flow_34"
model += f45 - 9.8*(theta4 - theta5) == 0, "dc_flow_45"
model += f56 - 5.6*(theta5 - theta6) == 0, "dc_flow_56"
model += f61 - 10.5*(theta6 - theta1) == 0, "dc_flow_61"


# Solve the model
model.solve()

print(f"Status: {pulp.LpStatus[model.status]}")
print(f"Optimal cost: {pulp.value(model.objective)}")
print(f"Generation p1: {p1.value()}")
print(f"Generation p2:{p2.value()}")
print(f"Generation p3: {p3.value()}")
print(f"Flows: f12={f12.value()}, f23={f23.value()}, f34={f34.value()}, f45={f45.value()}, f56={f56.value()}, f61={f61.value()}")

# Try to get dual values if your solver supports it
try:
    for name, constraint in model.constraints.items():
        if "balance" in name:
            print(f"Dual for {name}: {constraint.pi}")
except:
    print("Note: Getting dual values may require specific solver settings")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/robertsmith/PycharmProjects/electricity-optimization/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/sm/b4sn7nw913j9p3200tll43800000gn/T/9267b8b7faec4c078821e98c44896e0b-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/sm/b4sn7nw913j9p3200tll43800000gn/T/9267b8b7faec4c078821e98c44896e0b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 56 RHS
At line 70 BOUNDS
At line 95 ENDATA
Problem MODEL has 13 rows, 15 columns and 34 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-13) rows, 0 (-15) columns and 0 (-34) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 992.04365
After Postsolve, objective 992.04365, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 992.0436527 - 0 iterations ti